### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [5]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.Phases.PHASE_ORDER


In [6]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [8]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


random.seed(42)

model_phases = dr.Phases.PHASE_ORDER

label_type = "stai"
threshold = "fixed"

x, y = train.Train_WESAD.get_wesad_data(
    metrics, model_phases, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}
ensemble_weights = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.92902935 0.95514375 0.95113355 0.95775714 0.95533687 0.92380156
 0.95140606 0.94616406 0.95077739 0.94742406 0.92470488 0.9515781
 0.94920922 0.95318994 0.94923991 0.92935117 0.95522832 0.95939686
 0.96162931 0.95930035 0.93117648 0.95543963 0.96308571 0.9671958
 0.96177527        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 15], Predictions: [0 1], [65  4]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 15], Predictions: [0 1], [62  7]
Training RF ...
Model RF, Actual: [0 1], [54 15], Predictions: [0 1], [62  7]
Training XGB ...
Model XGB, Actual: [0 1], [54 15], Predictions: [0 1], [62  7]

None


[('bpm', 273), ('ecg_mean', 128), ('SCR_rate', 81), ('sdnn', 78), ('ecg_std', 78), ('rmssd', 72), ('hf_rr', 68), ('eda_mean', 66), ('ecg_var', 50), ('lf_hf_ratio', 45), ('eda_var

One or more of the test scores are non-finite: [0.84275789 0.88299287 0.88015927 0.90391667 0.89726723 0.84080233
 0.88120546 0.88391637 0.90417039 0.90741864 0.84740624 0.87870959
 0.88275654 0.90147406 0.89704048 0.89166428 0.89284192 0.89304063
 0.90511658 0.90926957 0.88257082 0.89339566 0.89305963 0.9013103
 0.90093183        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [56 14]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [55 15]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [55 15]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [55 15]

None


[('bpm', 309), ('ecg_std', 169), ('ecg_mean', 163), ('mean_SCL', 103), ('SCR_rate', 102), ('eda_mean', 98), ('rmssd', 95), ('eda_std', 92), ('ecg_var', 83), ('sdnn', 73), ('e

...

Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [68  2]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [63  7]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [65  5]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [63  7]

None


[('bpm', 317), ('ecg_std', 145), ('rmssd', 112), ('ecg_mean', 108), ('mean_SCL', 91), ('sdnn', 89), ('eda_mean', 84), ('ecg_median', 77), ('hf_rr', 74), ('SCR_rate', 67), 

One or more of the test scores are non-finite: [0.94661036 0.96356072 0.96652349 0.96942286 0.96392433 0.93526315
 0.95617015 0.9587303  0.96685003 0.96592684 0.94494993 0.95330511
 0.96144754 0.96840096 0.96745839 0.93639123 0.95344855 0.96250013
 0.96845883 0.96712599 0.93343587 0.95033519 0.96107717 0.96321954
 0.96281263        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [62  7], Predictions: [0 1], [68  1]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [62  7], Predictions: [0], [69]
Training RF ...
Model RF, Actual: [0 1], [62  7], Predictions: [0], [69]
Training XGB ...
Model XGB, Actual: [0 1], [62  7], Predictions: [0], [69]

None


[('bpm', 248), ('ecg_mean', 157), ('ecg_std', 70), ('eda_mean', 55), ('rmssd', 52), ('hf_rr', 42), ('eda_var', 42), ('sdnn', 41), ('SCR_rate', 35), ('lf_hf_ratio', 33), ('ecg_median', 32), ('ed

One or more of the test scores are non-finite: [0.88490694 0.90528616 0.9086554  0.90986186 0.90804928 0.87575514
 0.90147526 0.9091168  0.91126825 0.91396482 0.86981315 0.89965009
 0.91246327 0.91053471 0.90899912 0.87583431 0.90649253 0.91503042
 0.90753161 0.90679179 0.87498058 0.90291918 0.91068343 0.9042206
 0.90615531        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [52 18]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [52 18]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [50 20]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [54 16]

None


[('bpm', 292), ('ecg_mean', 184), ('ecg_std', 146), ('rmssd', 122), ('sdnn', 101), ('mean_SCL', 95), ('ecg_var', 89), ('lf_hf_ratio', 86), ('eda_mean', 85), ('ecg_median', 70)

In [9]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


random.seed(42)

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.1
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "weighted_avg"

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds, random_seed=15)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
weights = [ensemble_weights[model_name] for model_name in estimators.keys()]
weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.9855072463768116
	Precision: 1.0
	Recall: 0.875
	F1-score: 0.9333333333333333
	AUC score: 0.9375
----------------------------------------
